In [0]:
import os
import json

os.chdir("/content/drive/My Drive/Colab Notebooks/Code_search_engine")

In [0]:
!pip install transformers
!pip install tensorboardX

In [0]:
import torch

import transformers
from transformers import (WEIGHTS_NAME,BertConfig, BertForMaskedLM, BertTokenizer)

In [0]:
model_path = "./output/pytorch_model.bin"

config = BertConfig.from_json_file('./output/config.json')
config.output_hidden_states = True


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForMaskedLM.from_pretrained("bert-base-uncased",config=config)
model.load_state_dict(torch.load(model_path))
model.eval()

# Encode all the docstrings:

In [0]:
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange

from tensorboardX import SummaryWriter

import glob
import logging
import random
import json
import math
from utils import (convert_examples_to_features, output_modes, processors)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [34]:
df = pd.read_csv("./data/test.docstring",sep='\t', names=["docstring"])[:20000]
df.shape

(20000, 1)

In [36]:
outputs = []

for data in tqdm_notebook(df['docstring'],"Encoding: "):
  input_ids = torch.tensor(tokenizer.encode(data, add_special_tokens=True)).unsqueeze(0)
  outputs.append(model(input_ids, masked_lm_labels=input_ids)[2][-1].detach().numpy()[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
np.save('./docstrings_vecs.npy',outputs)

In [0]:
outputs = np.load('./docstrings_vecs.npy',allow_pickle=True)

In [7]:
from tqdm import tqdm_notebook, trange

docstrings_vecs = []

for embeddings in tqdm_notebook(outputs,"Encoding: "):
  size = embeddings.shape[0]
  sum_array = [sum(x) for x in zip(*embeddings)]
  avg_array = [sum_array[i]/size for i in range(len(sum_array))]
  docstrings_vecs.append(avg_array)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [0]:
np.save('./docstrings_avg_vecs.npy',docstrings_vecs)